## Implementing hyperparameter tuning

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
## multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



# read the data
path="../../Data Processing/Data/" # path to file relative to working directory
numerical = pd.read_csv(path+'numerical.csv')
categorical = pd.read_csv(path+'categorical.csv')
targets = pd.read_csv(path+'target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)

In [2]:
RAND_STATE = 42 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

In [3]:
data['TARGET_B'].value_counts() # distribution of target "B" (binary label)

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [4]:
# X,y
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis=1)
# split the data by type
numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

In [5]:
# onehot encoding (needed for SMOTE and sci-kit learn's random forest)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='if_binary').fit(categoricalX) #  drop the first category in each feature to reduce frame size
encoded_categorical = pd.DataFrame(encoder.transform(categoricalX).toarray()) # encode
X = pd.concat([numericalX, encoded_categorical], axis = 1) # rejoin
X.head(3)

TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  \
0      0  60.000000       5        9    0         0       39        34   
1      1  46.000000       6        9   16         0       15        55   
2      1  61.611649       3        1    2         0       20        29   

   WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  POP902  POP903  \
0        18        10         2       1        5     992     264     332   
1        11         6         2       1        9    3611     940     998   
2        33         6         8       1        1    7001    2040    2669   

   POP90C1  POP90C2  POP90C3  POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  \
0        0       35       65       47       53    92     1     0     0    11   
1       99        0        0       50       50    67     0     0    31     6   
2        0        2       98       49       51    96     2     0     0     2   

   ETH6  ETH7  ETH8  ETH9  ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  \
0     0     0     0     0      0      0      0     11      0      0      0   
1     4     2     6     4     14      0      0      2      0      1      4   
2     0     0     0     0      0      0      0      2      0      0      0   

   AGE901  AGE902  AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  \
0      39      48      51      40      50      54      25     31     42   
1      34      41      43      32      42      45      32     33     46   
2      35      43      46      37      45      49      23     35     40   

   CHIL3  AGEC1  AGEC2  AGEC3  AGEC4  AGEC5  AGEC6  AGEC7  CHILC1  CHILC2  \
0     27     11     14     18     17     13     11     15      12      11   
1     21     13     14     33     23     10      4      2      11      16   
2     25     13     20     19     16     13     10      8      15      14   

   CHILC3  CHILC4  CHILC5  HHAGE1  HHAGE2  HHAGE3  HHN1  HHN2  HHN3  HHN4  \
0      34      25      18      26      10      23    18    33    49    28   
1      36      22      15      12       1       5     4    21    75    55   
2      30      22      19      25      10      23    21    35    44    22   

   HHN5  HHN6  MARR1  MARR2  MARR3  MARR4  HHP1  HHP2  DW1  DW2  DW3  DW4  \
0    12     4     61      7     12     19   198   276   97   95    2    2   
1    23     9     69      4      3     24   317   360   99   99    0    0   
2     6     2     63      9      9     19   183   254   69   69    1    6   

   DW5  DW6  DW7  DW8  DW9   HV1   HV2  HV3  HV4  HU1  HU2  HU3  HU4  HU5  \
0    0    0    7    7    0   479   635    3    2   86   14   96    4    7   
1    0    0    0    0    0  5468  5218   12   10   96    4   97    3    9   
2    5    3    3    3    0   497   546    2    1   78   22   93    7   18   

   HHD1  HHD2  HHD3  HHD4  HHD5  HHD6  HHD7  HHD8  HHD9  HHD10  HHD11  HHD12  \
0    38    80    70    32    84    16     6     2     5      9     15      3   
1    59    94    88    55    95     5     4     1     3      5      4      2   
2    36    76    65    30    86    14     7     2     5     11     17      3   

   ETHC1  ETHC2  ETHC3  ETHC4  ETHC5  ETHC6  HVP1  HVP2  HVP3  HVP4  HVP5  \
0     17     50     25      0      0      0     2     7    13    27    47   
1     18     44      5      0      0      0    97    98    98    98    99   
2     17     60     18      0      1      0     0     1     6    18    50   

   HVP6  HUR1  HUR2  RHP1  RHP2  RHP3  RHP4  HUPA1  HUPA2  HUPA3  HUPA4  \
0     0     1    61    58    61    15     4      2      0      0     14   
1    94     0    83    76    73    21     5      0      0      0      4   
2     0     4    36    49    51    14     5      4      2     24     11   

   HUPA5  HUPA6  HUPA7  RP1  RP2  RP3  RP4     MSA    ADI    DMA   IC1   IC2  \
0      1      0      0    2    5   17   73     0.0  177.0  682.0   307   318   
1      0      0      0   91   91   91   94  4480.0   13.0  803.0  1088  1096   
2      2      3      6    0    2    9   44     0.0  281.0  

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TT_RATIO, random_state=RAND_STATE)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis=1)
X_test = X_test.drop(['TARGET_D'],   axis=1)

In [7]:
# again we need to drop the null (we use the same shuffling)
na_idx = X_train[X_train.isna().any(axis=1)].index
X_train = pd.DataFrame(X_train).drop(na_idx)
y_train = pd.DataFrame(y_train).drop(na_idx)

In [8]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)
y_train.value_counts()

TARGET_B
0           67903
1           67903
dtype: int64

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [50, 100,500],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']
    ##'max_samples' : ['None', 0.5],
    ##'max_depth':[3,5,10],
    ## 'bootstrap':[True,False]
    }
clf = RandomForestClassifier(random_state=RAND_STATE)

In [10]:
grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True,n_jobs=-1,)

In [11]:
grid_search.fit(X_train,y_train.values.ravel())

/Users/antares/.pyenv/versions/mambaforge-4.10.1-4/envs/ih_DA_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/antares/.pyenv/versions/mambaforge-4.10.1-4/envs/ih_DA_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/antares/.pyenv/versions/mambaforge-4.10.1-4/envs/ih_DA_env/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/antares/.pyenv/versions/mambaforge-4.10.1-4/envs/ih_DA_env/lib/python3.9/site-packages/sklearn/utils/validation

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_features': ['sqrt'], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [50, 100, 500]},
             return_train_score=True)

In [12]:
best_params = grid_search.best_params_ #To check the best set of parameters returned
best_params

{'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500}

In [13]:
pd.DataFrame(grid_search.cv_results_)

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       48.232684      1.154276         0.559298        0.100726   
1       88.627390      2.212101         0.935751        0.099336   
2      454.482473     13.119797         4.266331        0.171342   
3       47.724138      2.640214         0.518331        0.065678   
4       90.854703      2.200466         0.969913        0.064261   
5      469.180213     28.959546        11.798043       15.125033   
6       44.969731      1.001785         0.472277        0.024615   
7       88.949101      2.494044         0.892414        0.068782   
8      496.162207      9.476106         4.084767        0.317170   
9       55.918928     23.727436         0.464603        0.035474   
10      87.753903      3.014054         0.882916        0.042009   
11     365.707040      7.163086         2.705081        0.282441   

   param_max_features param_min_samples_leaf param_min_samples_split  \
0                sqrt                      1                       2   
1                sqrt                      1                       2   
2                sqrt                      1                       2   
3                sqrt                      1                       4   
4                sqrt                      1                       4   
5                sqrt                      1                       4   
6                sqrt                      2                       2   
7                sqrt                      2                       2   
8                sqrt                      2                       2   
9                sqrt                      2                       4   
10               sqrt                      2                       4   
11               sqrt                      2                       4   

   param_n_estimators                                             params  \
0                  50  {'max_features': 'sqrt', 'min_samples_leaf': 1...   
1                 100  {'max_features': 'sqrt', 'min_samples_leaf': 1...   
2                 500  {'max_features': 'sqrt', 'min_samples_leaf': 1...   
3                  50  {'max_features': 'sqrt', 'min_samples_leaf': 1...   
4                 100  {'max_features': 'sqrt', 'min_samples_leaf': 1...   
5                 500  {'max_features': 'sqrt', 'min_samples_leaf': 1...   
6                  50  {'max_features': 'sqrt', 'min_samples_leaf': 2...   
7                 100  {'max_features': 'sqrt', 'min_samples_leaf': 2...   
8                 500  {'max_features': 'sqrt', 'min_samples_leaf': 2...   
9                  50  {'max_features': 'sqrt', 'min_samples_leaf': 2...   
10                100  {'max_features': 'sqrt', 'min_samples_leaf': 2...   
11                500  {'max_features': 'sqrt', 'min_samples_leaf': 2...   

    split0_test_score  split1_test_score  split2_test_score  \
0            0.870223           0.998932           0.998417   
1            0.869818           0.998969           0.998711   
2            0.869229           0.999337           0.999116   
3            0.870886           0.999080           0.998895   
4            0.869892           0.999080           0.999006   
5            0.868935           0.999116           0.999116   
6            0.868971           0.998711           0.998527   
7            0.868382           0.998895           0.998785   
8            0.867536           0.999116           0.998859   
9            0.868971           0.998711           0.998527   
10           0.868382           0.998895           0.998785   
11           0.867536           0.999116           0.998859   

    split3_test_score  split4_test_score  mean_test_score  std_test_score  \
0            0.999080           0.998895         0.973109        0.051444   
1            0.999300           0.999080         0.973176        0.051679   
2            0.999632           0.999337         0.973330        0.052051   
3            0.999190           0.998564         0.973323        

<b> Please check RandomSearch as another algorithm comparable to GridSearch

In [14]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(random_state=RAND_STATE, **best_params)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=5)
print(np.mean(cross_val_scores))

0.9733303690654246


## Feature Importance

<b> Higher the score, the more important the feature is

In [15]:
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

In [16]:
len(X_train.columns)

360

In [17]:
feature_names = X_train.columns
feature_names = list(feature_names)

In [18]:
df = pd.DataFrame(list(zip(feature_names, clf.feature_importances_)))
df.columns = ['columns_name', 'score_feature_importance']
df.sort_values(by=['score_feature_importance'], ascending = False)

columns_name  score_feature_importance
349           19                  0.050890
348           18                  0.038808
343           13                  0.036900
344           14                  0.036894
350           20                  0.032789
..           ...                       ...
332            2                  0.000200
288         HC15                  0.000199
32         ETH12                  0.000101
319    ODATEW_MM                  0.000055
346           16                  0.000000

[360 rows x 2 columns]